In [77]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Project Day

### Importing the dataset

In [78]:
df=pd.read_csv("spam.csv",encoding='latin-1')
#dropping unrelated columns
df=df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])
#renaming columns
df=df.rename(columns={
     'v1':'label',
    'v2':'message'
}
)
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [79]:
#number of rows and columns
df.shape

(5572, 2)

### Data cleaning and preprocessing

In [80]:
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ExPertComputer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [81]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [82]:
ps=PorterStemmer()
lemmatizer=WordNetLemmatizer()

In [83]:
corpus=[]
for i in range(len(df)):
    review= re.sub('[^a-zA-Z]',' ',df['message'][i])
    review= review.lower()
    review= review.split()
    review= [ps.stem(word) for word in review if word not in set(stopwords.words("english"))]
    review= ' '.join(review)
    corpus.append(review)
    

### Creating bags of words model

X for message 

In [84]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2500,binary=True,ngram_range=(2,2))
X= cv.fit_transform(corpus).toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(5572, 2500))

y for labels

In [85]:
from sklearn.preprocessing import LabelEncoder
encoder= LabelEncoder()
df['label']=encoder.fit_transform(df['label'])

In [86]:
# y= pd.get_dummies(df['label'])
# y=y.iloc[:,1].values
y=df['label']

### Train,Test Split

In [87]:
from sklearn.model_selection import train_test_split

In [88]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.20,random_state=0)

In [89]:
X_train,y_train

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], shape=(4457, 2500)),
 1114    0
 3589    0
 3095    0
 1012    0
 3320    0
        ..
 4931    0
 3264    1
 1653    0
 2607    0
 2732    0
 Name: label, Length: 4457, dtype: int64)

### Model building

In [90]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model= MultinomialNB().fit(X_train,y_train)

### Prediction

In [91]:
y_pred= spam_detect_model.predict(X_test)

### Model evaluation

In [92]:
from sklearn.metrics import accuracy_score,classification_report

In [93]:
score= accuracy_score(y_test,y_pred)
print(score)

0.9641255605381166


In [94]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98       987
           1       0.77      0.99      0.86       128

    accuracy                           0.96      1115
   macro avg       0.88      0.98      0.92      1115
weighted avg       0.97      0.96      0.97      1115



### TF/IDF model

In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv=TfidfVectorizer(max_features=2500,ngram_range=(1,2))
X=tv.fit_transform(corpus).toarray()

### Train,Test,Split

In [96]:
from sklearn.model_selection import train_test_split

In [97]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.20,random_state=0)

In [98]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model= MultinomialNB().fit(X_train,y_train)

In [99]:
#prediction
y_pred=spam_detect_model.predict(X_test)

In [100]:
from sklearn.metrics import accuracy_score,classification_report

In [101]:
score= accuracy_score(y_test,y_pred)
print(score)

0.9704035874439462


In [102]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98       980
           1       0.81      0.99      0.89       135

    accuracy                           0.97      1115
   macro avg       0.90      0.98      0.94      1115
weighted avg       0.98      0.97      0.97      1115



### RandomForest

In [103]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()

In [104]:
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [105]:
y_pred=classifier.predict(X_test)

In [106]:
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

0.979372197309417
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       970
           1       0.87      0.99      0.93       145

    accuracy                           0.98      1115
   macro avg       0.93      0.99      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [107]:
!pip install joblib


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [108]:
import joblib

In [111]:
joblib.dump(classifier,"clasi_model.pkl")
joblib.dump(encoder,"label_encoder.pkl")
joblib.dump(tv, "vectorizer.pkl") 

['vectorizer.pkl']

In [110]:
!pip install streamlit


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
